In [43]:
import pandas as pd
import torch
import dgl
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import networkx as nx
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv
import time



In [44]:
nodes_df = pd.read_csv('clean_nodes.csv')
edges_df = pd.read_csv('clean_edges.csv')

1.Xây dựng đồ thị
# Tạo mapping từ profile_id sang index (0 đến N-1)
# Tạo tensor cho edge_index (dạng (2, num_edges))
# Tạo đồ thị DGL

In [45]:
profile_ids = nodes_df['profile_id'].tolist()
node_to_idx = {user: i for i, user in enumerate(profile_ids)}
# 2. Chuyển đổi danh sách cạnh thành tensor edge_index cho DGL
edge_list = []
for idx, row in edges_df.iterrows():
    src, tgt = row['source'], row['target']
    if src in node_to_idx and tgt in node_to_idx:
        edge_list.append( (node_to_idx[src], node_to_idx[tgt]) )


In [46]:
# Chuyển đổi edge_list thành tensor, với định dạng (source_indices, target_indices)
src_nodes, tgt_nodes = zip(*edge_list)
edge_array = torch.tensor(edge_list, dtype=torch.long).t()  # Kích thước [2, num_edges]

# Tạo đồ thị DGL
g = dgl.graph((edge_array[0], edge_array[1]), num_nodes=len(profile_ids))
print("Số lượng nút:", g.num_nodes(), "Số lượng cạnh:", g.num_edges())

Số lượng nút: 25669 Số lượng cạnh: 53452


2. Tính toán các chỉ số mạng cơ bản

In [47]:
nx_g = dgl.to_networkx(g,node_attrs=g.ndata,edge_attrs=g.edata)
print(nx_g.is_directed())
print("Đã chuyển sang NetworkX.")

# Nếu đồ thị lớn (n > 1000), dùng xấp xỉ cho betweenness (sample k nodes)
num_nodes = g.num_nodes()
k_sample = 100 if num_nodes > 1000 else None
in_degrees = nx.in_degree_centrality(nx_g)
out_degrees = nx.out_degree_centrality(nx_g)
start_time = time.time()
if k_sample:
    betweenness = nx.betweenness_centrality(nx_g, k=k_sample, normalized=True)
else:
    betweenness = nx.betweenness_centrality(nx_g, normalized=True)
print("Betweenness Centrality tính xong trong {:.2f} giây.".format(time.time()-start_time))

start_time = time.time()
# Closeness có thể tính trực tiếp (cẩn thận với đồ thị lớn)
closeness = nx.closeness_centrality(nx_g)
print("Closeness Centrality tính xong trong {:.2f} giây.".format(time.time()-start_time))

start_time = time.time()
# Eigenvector Centrality: nếu không hội tụ, bắt ngoại lệ và gán 0 cho nút đó
try:
    eigenvector = nx.eigenvector_centrality(nx_g)
except Exception as e:
    print("Eigenvector Centrality không hội tụ:", e)
    eigenvector = {i: 0.0 for i in range(num_nodes)}
print("Eigenvector Centrality tính xong trong {:.2f} giây.".format(time.time()-start_time))

# Tạo DataFrame chứa các chỉ số mạng (sắp xếp theo index của node)
metrics_df = pd.DataFrame({
    'profile_id': profile_ids,
    'in_degree': [in_degrees.get(node, 0) for node in range(num_nodes)],
    'out_degree': [out_degrees.get(node, 0) for node in range(num_nodes)],
    'betweenness': [betweenness.get(i, 0.0) for i in range(num_nodes)],
    'closeness': [closeness.get(i, 0.0) for i in range(num_nodes)],
    'eigenvector': [eigenvector.get(i, 0.0) for i in range(num_nodes)]
})
print("Các chỉ số mạng tính được, hiển thị 5 dòng đầu:")
print(metrics_df.head())

True
Đã chuyển sang NetworkX.
Betweenness Centrality tính xong trong 11.93 giây.
Closeness Centrality tính xong trong 266.24 giây.
Eigenvector Centrality không hội tụ: not implemented for multigraph type
Eigenvector Centrality tính xong trong 0.01 giây.
Các chỉ số mạng tính được, hiển thị 5 dòng đầu:
   profile_id  in_degree  out_degree  betweenness  closeness  eigenvector
0          14   0.002376    0.002104     0.017622   0.191469          0.0
1         512   0.000000    0.000039     0.000000   0.000000          0.0
2         533   0.000039    0.000039     0.000000   0.183487          0.0
3         534   0.126695    0.110059     0.155894   0.254787          0.0
4         564   0.004285    0.000117     0.000387   0.194558          0.0


In [48]:
metrics_df[metrics_df['in_degree'] > 0]

,profile_id,in_degree,out_degree,betweenness,closeness,eigenvector
0,14,0.002376,0.002104,0.017622,0.191469,0.0
2,533,0.000039,0.000039,0.000000,0.183487,0.0
3,534,0.126695,0.110059,0.155894,0.254787,0.0
4,564,0.004285,0.000117,0.000387,0.194558,0.0
5,598,0.000078,0.000117,0.000119,0.174467,0.0
...,...,...,...,...,...,...
25664,11313784,0.000039,0.000000,0.000000,0.129857,0.0
25665,11313915,0.000039,0.000000,0.000000,0.134640,0.0
25666,11313924,0.000039,0.000000,0.000000,0.134640,0.0
25667,11313953,0.000039,0.000000,0.000000,0.134640,0.0


# 3. Nhúng đồ thị sử dụng GNN với DGL (GCN)


In [49]:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_feats)
        self.conv2 = GraphConv(hidden_feats, out_feats)
    
    def forward(self, graph, features):
        x = self.conv1(graph, features)
        x = F.relu(x)
        x = self.conv2(graph, x)
        return x

In [50]:
# Nếu chưa có đặc trưng cho nút, tạo feature ngẫu nhiên (ví dụ, kích thước 16)
feat_dim = 16
if 'feat' not in g.ndata:
    g.ndata['feat'] = torch.randn(g.num_nodes(), feat_dim)

embedding_dim = 16  # Kích thước vector nhúng mong muốn
model = GCN(feat_dim, 2 * embedding_dim, embedding_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)


In [51]:
# Để giảm chi phí tính toán, ta huấn luyện số epoch thấp (ví dụ 100 epoch)
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    g = dgl.add_self_loop(g)
    # Ở đây, làm bài toán tự giám sát đơn giản: tái tạo lại đặc trưng ban đầu bằng MSE
    z = model(g, g.ndata['feat'])
    loss = F.mse_loss(z, g.ndata['feat'])
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:03d}: Loss = {loss.item():.4f}")

# Sau huấn luyện, trích xuất vector nhúng của các nút
model.eval()
with torch.no_grad():
    node_embeddings = model(g, g.ndata['feat']).cpu().numpy()

# Tạo DataFrame cho embedding: đặt tên cột là gnn_emb_0, gnn_emb_1, ... gnn_emb_63
embedding_df = pd.DataFrame(node_embeddings, columns=[f'gnn_emb_{i}' for i in range(embedding_dim)])
embedding_df['profile_id'] = profile_ids

Epoch 010: Loss = 0.5757
Epoch 020: Loss = 0.2671
Epoch 030: Loss = 0.1733
Epoch 040: Loss = 0.1237
Epoch 050: Loss = 0.0936
Epoch 060: Loss = 0.0711
Epoch 070: Loss = 0.0547
Epoch 080: Loss = 0.0421
Epoch 090: Loss = 0.0324
Epoch 100: Loss = 0.0250


In [52]:
##############################################
# 4. Kết hợp đặc trưng: Metrics + GNN Embedding
##############################################
# Chuẩn hóa các chỉ số centrality
scaler_centrality = MinMaxScaler()
centrality_columns = ['in_degree', 'out_degree', 'betweenness', 'closeness', 'eigenvector']
metrics_df[centrality_columns] = scaler_centrality.fit_transform(metrics_df[centrality_columns])

# Chuẩn hóa node embeddings
scaler_embedding = StandardScaler()
embedding_columns = [f'gnn_emb_{i}' for i in range(embedding_dim)]
embedding_df[embedding_columns] = scaler_embedding.fit_transform(embedding_df[embedding_columns])

combined_df = pd.merge(metrics_df, embedding_df, on='profile_id')
print("Kết quả đặc trưng kết hợp, 5 dòng đầu:")
print(combined_df.head())

# Lưu kết quả ra file CSV
combined_df.to_csv('combined_user_features.csv', index=False)
print("Đã lưu đặc trưng kết hợp vào file 'combined_user_features.csv'")

Kết quả đặc trưng kết hợp, 5 dòng đầu:
   profile_id  in_degree  out_degree  betweenness  closeness  eigenvector  \
0          14   0.018758    0.019115     0.113038   0.739306          0.0   
1         512   0.000000    0.000354     0.000000   0.000000          0.0   
2         533   0.000308    0.000354     0.000000   0.708484          0.0   
3         534   1.000000    1.000000     1.000000   0.983790          0.0   
4         564   0.033825    0.001062     0.002480   0.751234          0.0   

   gnn_emb_0  gnn_emb_1  gnn_emb_2  gnn_emb_3  ...  gnn_emb_6  gnn_emb_7  \
0   0.588825   0.032355   0.828369  -0.817316  ...  -0.125660   0.303268   
1   0.632943  -0.306380  -0.133208  -1.507565  ...   0.226926   1.363894   
2  -0.165315  -0.741651   0.762098  -1.252277  ...  -1.088730   0.131084   
3  -0.675118   0.867779   0.137951   0.349359  ...   0.243103  -0.655177   
4  -0.236927  -0.497405   0.383309   0.069869  ...  -0.293359   0.401449   

   gnn_emb_8  gnn_emb_9  gnn_emb_10  gnn_